In [ ]:
import os
import numpy as np
import pandas as pd
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
from BuildDatabase import BuildDatabase

pd.options.plotting.backend = 'plotly'

# repo_dir = '/home/cam/Documents/database_tools/'
# os.chdir(repo_dir)

worker = BuildDatabase(
    output_dir='data-2022-09-23/',
    win_len=1024,
    fs=125,
    data_dir='physionet.org/files/mimic3wdb/1.0/',
)
windows, n_excluded, sim, snr, hr, abp_max, abp_min = worker.run()

In [ ]:
import os
import pandas as pd
from SignalProcessor import SignalProcessor

pd.options.plotting.backend = 'plotly'

# repo_dir = '/home/cam/Documents/database_tools/'
# os.chdir(repo_dir)

worker = SignalProcessor(
    files=['physionet.org/files/mimic3wdb/1.0/30/3000480/3000480_0018'],
    win_len=1024,
    fs=125,
)

windows, sim, snr, hr = worker.run(
    sim1=0.5,
    sim2=0.8,
    snr_t=20,
    hr_diff=1/6,
    f0_low=0.667,
    f0_high=3.0
)
len(windows)

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import numpy as np

x, y = sim[:, 0], sim[:, 1]
n = 100

fig = plt.figure(figsize=(8, 6), dpi=150)
ax = fig.add_subplot(111, projection='3d')
hist, xedges, yedges = np.histogram2d(x, y, bins=n, range=[[0, 1], [0, 1]])

xpos, ypos = np.meshgrid(xedges[:-1] + 1/n, yedges[:-1] + 1/n, indexing='ij')
xpos = xpos.flatten('F')
ypos = ypos.flatten('F')
zpos = 0

dx = .01 * np.ones_like(zpos)
dy = dx.copy()
dz = hist.ravel()

mask_dz = dz == 0
ax.bar3d(xpos[~mask_dz], ypos[~mask_dz], zpos, dx, dy, dz[~mask_dz], color='b', zsort='average')

plt.show()

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from Preprocessing.SignalLevelFiltering import get_snr

i = 17000
x, y = windows[i][0], windows[i][1]

scaler = MinMaxScaler(feature_range=(60, 120))
x = scaler.fit_transform(x.reshape(-1, 1)).reshape(-1,)

df = pd.DataFrame(
    dict(
        pleth=x,
        abp=y,
    )
)

df.plot()

In [ ]:
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from neurokit2.ppg import ppg_findpeaks
from heartpy.preprocessing import flip_signal

def segment_beats(x, fs=125):
    x_flip = flip_signal(x)
    idx = ppg_findpeaks(x_flip, sampling_rate=fs)['PPG_Peaks']
    return idx

i = 20000
x, y = windows[i][0], windows[i][1]

idx_plotx = segment_beats(x)
idx_x = np.concatenate(
    [
        np.array(idx_plotx[0:-1]),
        np.array(idx_plotx[1:]),
    ]
).reshape(-1, 2, order='F')
idx_x = np.concatenate((np.array([0, idx_x[0, 0]]).reshape(-1, 2), idx_x))

idx_ploty = segment_beats(y)
idx_y = np.concatenate(
    [
        np.array(idx_ploty[0:-1]),
        np.array(idx_ploty[1:]),
    ]
).reshape(-1, 2, order='F')
idx_y = np.concatenate((np.array([0, idx_y[0, 0]]).reshape(-1, 2), idx_y))

trace1 = go.Scatter(
    x=np.arange(0, len(x), 1),
    y=x,
    name='',
    marker=dict(color='rgb(34,163,192)'),
)

trace2 = go.Scatter(
    x=idx_plotx,
    y=x[idx_plotx],
    name='Minima',
    mode='markers',
)

trace3 = go.Scatter(
    x=np.arange(0, len(x), 1),
    y=y,
    name='',
    marker=dict(color='rgb(34,163,192)'),
)

trace4 = go.Scatter(
    x=idx_ploty,
    y=y[idx_ploty],
    name='Minima',
    mode='markers',
)

fig = make_subplots(rows=1, cols=1)
fig.add_trace(trace1, row=1, col=1)
fig.add_trace(trace2, row=1, col=1)
fig.add_trace(trace3, row=1, col=1)
fig.add_trace(trace4, row=1, col=1)

fig.show()

In [ ]:
import numpy as np
from tqdm.notebook import tqdm
from neurokit2.ppg import ppg_findpeaks
from heartpy.preprocessing import flip_signal

def segment_beats(x, fs=125):
    x_flip = flip_signal(x)
    idx = ppg_findpeaks(x_flip, sampling_rate=fs)['PPG_Peaks']
    return idx

def get_similarity(x, y):
    """
    Calculates time or spectral similarity.

    Args:
        x (np.ndarray): PLETH data.
        y (np.ndarray): ABP data.
        spectral (boolean): If True calculate fft
            of signals.

    Returns:
        coef (float): Pearson correlation coefficient.
    """
    diff = len(x) - len(y)
    if diff < 0:
        y = y[0:diff]
    elif diff > 0:
        x = x[0:-1 * diff]


    x_bar = np.mean(x)
    y_bar = np.mean(y)
    y_temp = y - y_bar
    x_temp = x - x_bar
    covar = np.sum( (x_temp * y_temp) )
    var = np.sqrt( np.sum( (x_temp ** 2 ) ) * np.sum( (y_temp ** 2) ) )
    coef = covar / var
    return coef

similarities = []
for win in tqdm(windows[:, 0], total=len(windows[:, 0])):
    idx = segment_beats(win)
    idx = np.concatenate(
        [
            np.array(idx[0:-1]),
            np.array(idx[1:]),
        ]
    ).reshape(-1, 2, order='F')
    if len(idx) == 0:
        continue
    else:
        idx = np.concatenate((np.array([0, idx[0, 0]]).reshape(-1, 2), idx))

        sim = []
        for i, j in idx:
            beat = win[i:j]
            try:
                sim.append(
                    get_similarity(x=last_beat,
                                y=beat)
                )
            except NameError:
                pass
            last_beat = beat
        sim.append(
            get_similarity(x=last_beat,
                        y=win[j:])
        )
        similarities.append(np.mean(sim))
